In [1]:
from dataset import dataset
from attention import Attention2D
import utils as utils
import datetime
import torch
import torch.nn as nn
from os import mkdir, getcwd
from os.path import join, dirname

In [3]:
rot, t = utils.camTrans()
model = Attention2D() #.to(device)
model_path = '../checkpoints/spatial-softmax/20210715-175329/checkpoint-2000.pth'
md = torch.load(model_path)
ts = md['model']

In [11]:
# model.load_state_dict(md['model'])

In [2]:
import numpy as np
import math

def euler2rotm(theta):
    R_x = np.array([[1,         0,                  0                   ],
                    [0,         math.cos(theta[0]), -math.sin(theta[0]) ],
                    [0,         math.sin(theta[0]), math.cos(theta[0])  ]
                    ])
    R_y = np.array([[math.cos(theta[1]),    0,      math.sin(theta[1])  ],
                    [0,                     1,      0                   ],
                    [-math.sin(theta[1]),   0,      math.cos(theta[1])  ]
                    ])         
    R_z = np.array([[math.cos(theta[2]),    -math.sin(theta[2]),    0],
                    [math.sin(theta[2]),    math.cos(theta[2]),     0],
                    [0,                     0,                      1]
                    ])            
    R = np.dot(R_z, np.dot( R_y, R_x ))
    return R

In [50]:
R = euler2rotm(np.random.rand(3))
camera_intrinsic=np.array([[450, 0 , 320], [0, 450, 240], [0, 0, 1]])
ifx = 32/45
ify = 24/45
pos_x, pos_y = np.meshgrid(
        np.linspace(-1., 1., 240),
        np.linspace(-1., 1., 320)
        )
pos_x *= ifx
pos_y *= ify
print(R, camera_intrinsic)

[[ 0.85617374 -0.27798709  0.43553381]
 [ 0.31443745  0.94919877 -0.01227939]
 [-0.40999464  0.14746143  0.90008862]] [[450   0 320]
 [  0 450 240]
 [  0   0   1]]


In [62]:
class TransformNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Conv2d(3, 3, 1)
        
    def forward(self, data):
        return self.l1(data)
    
class BackProjectionNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Conv2d(2,2,1)
        pos_x, pos_y = np.meshgrid(
                np.linspace(-1., 1., self.height),
                np.linspace(-1., 1., self.width)
                )
        pos_x = torch.Tensor(pos_x).float()
        pos_y = torch.Tensor(pos_y).float()
        self.register_buffer('pos_x', pos_x)
        self.register_buffer('pos_y', pos_y)
        self.xy = torch.cat((pos_x[None,:], pos_y[None, :]), 0)
        self.ones = torch.ones(height, width)
        self.l1 = nn.Conv2d(2, 2, 1)
        self.l2 = nn.Conv2d(3, 3, 1)
    def forward(self, data):
        return self.l1(data)

class CoordinateTransformer(nn.Module):
    
    def __init__(self, height, width):
#         super(CoordinateTransformer).__init__()
        super().__init__()
        
        pos_x, pos_y = np.meshgrid(
                np.linspace(-1., 1., height),
                np.linspace(-1., 1., width)
                )
        pos_x = torch.Tensor(pos_x).float()
        pos_y = torch.Tensor(pos_y).float()
#         self.register_buffer('pos_x', pos_x)
#         self.register_buffer('pos_y', pos_y)
        self.xy = torch.cat((pos_x[None,:], pos_y[None, :]), 0)
        self.ones = torch.ones(height, width)
        self.l1 = nn.Conv2d(2, 2, 1)
        self.l2 = nn.Conv2d(3, 3, 1)
        
    def forward(self, depth):
        output = self.l1(self.xy)
        output = torch.cat((output, self.ones[None, :]), 0)
        output = output * depth
        output = self.l2(output)
        return output

In [76]:
# data_len = 2000000
# def dataset(data_len):
#     data = torch.rand((data_len,3,1,1))
#     Rot = torch.Tensor(R)
#     reshape_data = torch.transpose(torch.squeeze(data), 1, 0)
# #     print(reshape_data.shape)
#     output = torch.matmul(Rot, reshape_data)
#     output = torch.transpose(output, 1, 0).reshape((data_len,3,1,1))
#     return data, output
# #     print(output.shape)

class dataset:
    def __init__(self, height=240,width=320):
        self.R = np.eye(3)
#         camera_intrinsic=np.array([[450, 0 , 320], [0, 450, 240], [0, 0, 1]])
        ifx = 32/45
        ify = 24/45
        self.height = height
        self.width = width
        pos_x, pos_y = np.meshgrid(
                np.linspace(-1., 1., width),
                np.linspace(-1., 1., height)
                )
        pos_x = torch.Tensor(pos_x).float()
        pos_y = torch.Tensor(pos_y).float()
        pos_x *= ifx
        pos_y *= ify
        ones = torch.ones(height,width)
        
#         print(pos_x.shape, pos_y.shape, ones.shape)
        
        self.base = torch.cat((pos_x[None,:], pos_y[None,:], ones[None, :]), 0)
        
        self.base.reshape(1, 3, height, width)

        
    def RandomRot(self):
        self.R = torch.Tensor(self.euler2rotm(np.random.rand(3)))
        
    def euler2rotm(self,theta):
        R_x = np.array([[1,         0,                  0                   ],
                        [0,         math.cos(theta[0]), -math.sin(theta[0]) ],
                        [0,         math.sin(theta[0]), math.cos(theta[0])  ]
                        ])
        R_y = np.array([[math.cos(theta[1]),    0,      math.sin(theta[1])  ],
                        [0,                     1,      0                   ],
                        [-math.sin(theta[1]),   0,      math.cos(theta[1])  ]
                        ])         
        R_z = np.array([[math.cos(theta[2]),    -math.sin(theta[2]),    0],
                        [math.sin(theta[2]),    math.cos(theta[2]),     0],
                        [0,                     0,                      1]
                        ])            
        R = np.dot(R_z, np.dot( R_y, R_x ))
        return R
    
    def get(self, length):
        depth = torch.randn(length, 1, self.height, self.width)
        pc = self.base * depth
        print(pc.shape)
        pc = torch.matmul(self.R, pc)
        return depth, pc

In [77]:
model = CoordinateTransformer(240,320)

learning_rate = 1e-4
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,momentum=0.9)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20000,30000,60000,80000], gamma=0.1)

data = dataset()
data.RandomRot()

loss_fn = torch.nn.MSELoss(reduction='sum')
for i in range(500):
    x, y_pred = data.get(5000)
    y = model(x)
    loss = loss_fn(y, y_pred)
#     print(i, loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    x_val, y_val = data.get(5000)
    y_h = model(x_val)
    loss_val = loss_fn(y_h, y_val)
    print(i, 'train: ', loss.item(), ', val: ', loss_val.item())


torch.Size([5000, 3, 240, 320])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (4800000x240 and 3x3)

In [29]:
torch.ones(4,3)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

In [34]:
pos_x, pos_y = np.meshgrid(
                np.linspace(-1., 1., 31),
                np.linspace(-1., 1., 21)
                )
print(pos_x.shape)
pos_x = torch.Tensor(pos_x).float()
pos_y = torch.Tensor(pos_y).float()

(21, 31)


In [40]:
tx = torch.cat((pos_x[None,:], pos_y[None, :]), 0)
ones = torch.ones(21, 31)
output = torch.cat((tx, ones[None,:]), 0)
print(output.shape)

torch.Size([3, 21, 31])


In [41]:
z = torch.rand(1, 21, 31)
pc = output * z
print(pc.shape)

torch.Size([3, 21, 31])


In [42]:
txy = torch.ones((3,3,4))
tz = torch.ones((1,3,4)) * 5
print(txy, tz)
print(txy * tz)

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]]) tensor([[[5., 5., 5., 5.],
         [5., 5., 5., 5.],
         [5., 5., 5., 5.]]])
tensor([[[5., 5., 5., 5.],
         [5., 5., 5., 5.],
         [5., 5., 5., 5.]],

        [[5., 5., 5., 5.],
         [5., 5., 5., 5.],
         [5., 5., 5., 5.]],

        [[5., 5., 5., 5.],
         [5., 5., 5., 5.],
         [5., 5., 5., 5.]]])


In [52]:
a = torch.ones(2,1,3,4)
b = torch.ones(1,3,3,4) * 2
c = a * b
print(c.shape,c)

torch.Size([2, 3, 3, 4]) tensor([[[[2., 2., 2., 2.],
          [2., 2., 2., 2.],
          [2., 2., 2., 2.]],

         [[2., 2., 2., 2.],
          [2., 2., 2., 2.],
          [2., 2., 2., 2.]],

         [[2., 2., 2., 2.],
          [2., 2., 2., 2.],
          [2., 2., 2., 2.]]],


        [[[2., 2., 2., 2.],
          [2., 2., 2., 2.],
          [2., 2., 2., 2.]],

         [[2., 2., 2., 2.],
          [2., 2., 2., 2.],
          [2., 2., 2., 2.]],

         [[2., 2., 2., 2.],
          [2., 2., 2., 2.],
          [2., 2., 2., 2.]]]])
